In [81]:
import numpy as np
from random import randint
from datetime import datetime
from buscaminas import MineSweeper as ms
from datasets import DataSets as ds

from keras.models import model_from_json

In [82]:
from importlib import reload
import buscaminas
reload(buscaminas)
from buscaminas import MineSweeper as ms
from importlib import reload
import datasets
reload(datasets)
from datasets import DataSets as ds


In [83]:
game = ms(5,5,3,neverends=True)

In [84]:
model = create_cnn_model(5,5)

(None, 5, 5, 1)
(None, 5, 5, 64)
(None, 3, 3, 64)
(None, 1, 1, 64)
(None, 64)
(None, 25)
(None, 25)


In [75]:
a,b = data.get_data(model,data_size=5)

ValueError: could not broadcast input array from shape (5,5,1) into shape (5)

In [85]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.layers.core import Dense, Activation
from keras.layers.advanced_activations import PReLU
from keras.optimizers import SGD , Adam, RMSprop
def create_cnn_model(alto,ancho):
    im_shape = (alto,ancho,1)
    model = Sequential()
    print((None,*im_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=im_shape))
    print(model.output_shape)
    model.add(Conv2D(64, (3, 3), activation='relu'))
    print(model.output_shape)
    model.add(MaxPool2D((2, 2)))
    print(model.output_shape)
    #model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    print(model.output_shape)
    #model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    #print(model.output_shape)
    #model.add(Conv2D(128, (3, 3), activation='relu'))
    #print(model.output_shape)
    model.add(Dense(alto*ancho))
    print(model.output_shape)
    model.add(PReLU())
    model.add(Dense(alto*ancho))
    model.add(PReLU())
    model.add(Dense(alto*ancho))
    print(model.output_shape)
    model.compile(optimizer='adam', loss='mse')
    return model

def load_model():
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    return loaded_model

In [95]:
alto = 5
ancho = 5
bombas = 3
model = create_cnn_model(alto,ancho)
data = ds(gamma=.95,max_memory=1000)
#model = load_model()
#model.compile(optimizer='adam', loss='mse')

(None, 5, 5, 1)
(None, 5, 5, 64)
(None, 3, 3, 64)
(None, 1, 1, 64)
(None, 64)
(None, 25)
(None, 25)


In [96]:
m_train(model,data,alto,ancho,bombas,epsilon=.1,cant_juegos=1,cant_mov_entrenar=100,last_ones=500,verbose=True)

Partida numero: 0 con 0 puntos y 118 turnos, 10.3 seconds y perdio


In [89]:
from importlib import reload
import buscaminas
reload(buscaminas)
from buscaminas import MineSweeper as ms
from importlib import reload
import datasets
reload(datasets)
from datasets import DataSets as ds

In [90]:
# This is a small utility for printing readable time strings:
def format_time(seconds):
    if seconds < 400:
        s = float(seconds)
        return "%.1f seconds" % (s,)
    elif seconds < 4000:
        m = seconds / 60.0
        return "%.2f minutes" % (m,)
    else:
        h = seconds / 3600.0
        return "%.2f hours" % (h,)


In [94]:
def m_train(model,data,alto,ancho,bombas,epsilon=.1,cant_juegos=1,cant_mov_entrenar=10,last_ones=10,verbose=False):    
    start_time = datetime.now()
    save = 0
    for juego in range(cant_juegos):
        #Creo Juego        
        game = ms(alto,ancho,bombas,True)
        duracion = 0
        cant_turnos = 0
        resultado = "gano"
        bombas_click=0
        mal_click=0
        bien_click=0
        while not game.gameover:
            #Saco los bordes
            #este_turno = game.matriz_usuario.reshape(1,-1)  
            este_turno = game.matriz_usuario.reshape(1,alto,ancho,1).copy()

            # Genereo proximo movimiento
            al_azar = (np.random.rand() <= epsilon)
            if not al_azar:                
                action = np.argmax(model.predict(este_turno)[0])
            else:
                action = randint(0,alto*ancho-1)

            t1,t2 = game.get_loc(action) 
            cant = game.click(t1,t2)
            
            if game.gameover:                
                reward = 100            
            elif cant==-1:
                reward = -10
                resultado="perdio"
                bombas_click+=1
            elif cant==0:
                reward = -1
                mal_clicks+=1
            else:
                reward = 1
                bien_clicks+=1
            episode = [este_turno, 
                       action, 
                       reward,
                       game.matriz_usuario.reshape(1,alto,ancho,-1),
                       game.gameover]
            data.remember(episode)
            cant_turnos += 1

            # Train neural network model
            inputs, targets = data.get_data(model,data_size=cant_mov_entrenar,last_few=last_ones)        
            h = model.fit(inputs,targets,epochs=8,batch_size=32,verbose=0)
            
        dt = datetime.now() - start_time
        t = format_time(dt.total_seconds())            

        with open("progreso_bot.txt", "a+") as f:
            f.write(f"{bombas_click},{mal_click},{bien_click},{resultado}\n")

        if verbose:
            print(f"Partida numero: {juego} con {bombas_click},{mal_click},{bien_click},{resultado} duro {t} y {resultado}")    
    if save==50:
        save = 0
        # serialize model to JSON
        model_json = model.to_json()
        with open("model.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model.save_weights("model.h5")
        print("Saved model to disk")
    else:
        save+=1
        

# This is a small utility for printing readable time strings:
def format_time(seconds):
    if seconds < 400:
        s = float(seconds)
        return "%.1f seconds" % (s,)
    elif seconds < 4000:
        m = seconds / 60.0
        return "%.2f minutes" % (m,)
    else:
        h = seconds / 3600.0
        return "%.2f hours" % (h,)
